to deploy the end-to-end pipeline to generate prediction results for new data from trained TF model

In [1]:
!pwd

/workspace/merlin/hugectr_inference_backend/hps_backend/examples


In [1]:
# define some data folder to store the model related files
# Standard Libraries
import os
from time import time
import re
import shutil
import glob
import warnings

BASE_DIR = "/wdl_infer"
model_folder  = os.path.join(BASE_DIR, "model")
wdl_model_repo= os.path.join(model_folder, "wdl")
wdl_version =os.path.join(wdl_model_repo, "1")

if os.path.isdir(model_folder):
    shutil.rmtree(model_folder)
os.makedirs(model_folder)

if os.path.isdir(wdl_model_repo):
    shutil.rmtree(wdl_model_repo)
os.makedirs(wdl_model_repo)

if os.path.isdir(wdl_version):
    shutil.rmtree(wdl_version)
os.makedirs(wdl_version)

In [2]:
!cp -r /workspace/data/wdl_models/wdl0_sparse_20000.model $wdl_version/
!cp -r /workspace/data/wdl_models/wdl1_sparse_20000.model $wdl_version/
!cp /workspace/data/wdl_models/wdl_dense_20000.model $wdl_version/
!cp /workspace/data/wdl_models/wdl.json $wdl_version/
!ls -l $wdl_version

total 5840
-rwxr-xr-x 1 root root    3590 Jun 29 07:55 wdl.json
drwxr-xr-x 2 root root    4096 Jun 29 07:55 wdl0_sparse_20000.model
drwxr-xr-x 2 root root    4096 Jun 29 07:55 wdl1_sparse_20000.model
-rw-r--r-- 1 root root 5963780 Jun 29 07:55 wdl_dense_20000.model


In [3]:
%%writefile $wdl_model_repo/config.pbtxt
name: "wdl"
backend: "hugectr"
max_batch_size:64,
input [
   {
    name: "DES"
    data_type: TYPE_FP32
    dims: [ -1 ]
  },
  {
    name: "CATCOLUMN"
    data_type: TYPE_INT64
    dims: [ -1 ]
  },
  {
    name: "ROWINDEX"
    data_type: TYPE_INT32
    dims: [ -1 ]
  }
]
output [
  {
    name: "OUTPUT0"
    data_type: TYPE_FP32
    dims: [ -1 ]
  }
]
instance_group [
  {
    count: 1
    kind : KIND_GPU
    gpus:[0]
  }
]

parameters [
  {
  key: "config"
  value: { string_value: "/wdl_infer/model/wdl/1/wdl.json" }
  },
  {
  key: "gpucache"
  value: { string_value: "true" }
  },
  {
  key: "hit_rate_threshold"
  value: { string_value: "0.8" }
  },
  {
  key: "gpucacheper"
  value: { string_value: "0.5" }
  },
  {
  key: "label_dim"
  value: { string_value: "1" }
  },
  {
  key: "slots"
  value: { string_value: "28" }
  }
]

Writing /wdl_infer/model/wdl/config.pbtxt


In [4]:
%%writefile /wdl_infer/model/ps.json
{
    "supportlonglong":true,
    "models":[
        {
            "model":"wdl",
            "sparse_files":["/wdl_infer/model/wdl/1/wdl0_sparse_20000.model", "/wdl_infer/model/wdl/1/wdl1_sparse_20000.model"],
            "dense_file":"/wdl_infer/model/wdl/1/wdl_dense_20000.model",
            "network_file":"/wdl_infer/model/wdl/1/wdl.json",
            "num_of_worker_buffer_in_pool": 4,
            "num_of_refresher_buffer_in_pool":1,
            "deployed_device_list":[0],
            "max_batch_size":64,
            "default_value_for_each_table":[0.0,0.0],
            "hit_rate_threshold":0.9,
            "gpucacheper":0.5,
            "gpucache":true,
            "cache_refresh_percentage_per_iteration":0.2,
            "maxnum_des_feature_per_sample": 13,
            "maxnum_catfeature_query_per_table_per_sample" : [2,26],
            "embedding_vecsize_per_table" : [1,15],
            "slot_num":28
        }
    ]  
}

Writing /wdl_infer/model/ps.json


In [5]:
!ls  -l $wdl_model_repo
!ls -l $wdl_version

total 8
drwxr-xr-x 4 root root 4096 Jun 29 07:55 1
-rw-r--r-- 1 root root  838 Jun 29 07:55 config.pbtxt
total 5840
-rwxr-xr-x 1 root root    3590 Jun 29 07:55 wdl.json
drwxr-xr-x 2 root root    4096 Jun 29 07:55 wdl0_sparse_20000.model
drwxr-xr-x 2 root root    4096 Jun 29 07:55 wdl1_sparse_20000.model
-rw-r--r-- 1 root root 5963780 Jun 29 07:55 wdl_dense_20000.model


In [9]:
!tritonserver --model-repository=/wdl_infer/model/ --load-model=wdl --model-control-mode=explicit --backend-directory=/usr/local/hugectr/backends --backend-config=hugectr,ps=/wdl_infer/model/ps.json

I0629 05:21:03.925945 10669 pinned_memory_manager.cc:240] Pinned memory pool is created at '0x7f6f16000000' with size 268435456
I0629 05:21:03.942581 10669 cuda_memory_manager.cc:105] CUDA memory pool is created on device 0 with size 67108864
I0629 05:21:03.942590 10669 cuda_memory_manager.cc:105] CUDA memory pool is created on device 1 with size 67108864
I0629 05:21:03.942595 10669 cuda_memory_manager.cc:105] CUDA memory pool is created on device 2 with size 67108864
I0629 05:21:03.942599 10669 cuda_memory_manager.cc:105] CUDA memory pool is created on device 3 with size 67108864
I0629 05:21:03.942603 10669 cuda_memory_manager.cc:105] CUDA memory pool is created on device 4 with size 67108864
I0629 05:21:03.942607 10669 cuda_memory_manager.cc:105] CUDA memory pool is created on device 5 with size 67108864
I0629 05:21:03.942611 10669 cuda_memory_manager.cc:105] CUDA memory pool is created on device 6 with size 67108864
I0629 05:21:03.942615 10669 cuda_memory_manager.cc:105] CUDA memory

I0602 02:59:54.707962 549 pinned_memory_manager.cc:240] Pinned memory pool is created at '0x7f46de000000' with size 268435456
I0602 02:59:54.725485 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 0 with size 67108864
I0602 02:59:54.725493 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 1 with size 67108864
I0602 02:59:54.725501 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 2 with size 67108864
I0602 02:59:54.725507 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 3 with size 67108864
I0602 02:59:54.725512 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 4 with size 67108864
I0602 02:59:54.725519 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 5 with size 67108864
I0602 02:59:54.725524 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 6 with size 67108864
I0602 02:59:54.725531 549 cuda_memory_manager.cc:105] CUDA memory pool is created on device 7 with size 67108864
I0602 02:59:55.799304 549 model_repository_manager.cc:997] loading: wdl:1
I0602 02:59:55.921125 549 hugectr.cc:1597] TRITONBACKEND_Initialize: hugectr
I0602 02:59:55.921155 549 hugectr.cc:1604] Triton TRITONBACKEND API version: 1.8
I0602 02:59:55.921161 549 hugectr.cc:1608] 'hugectr' TRITONBACKEND API version: 1.8
I0602 02:59:55.921166 549 hugectr.cc:1631] The HugeCTR backend Repository location: /usr/local/hugectr/backends/hugectr
I0602 02:59:55.921171 549 hugectr.cc:1640] The HugeCTR backend configuration: {"cmdline":{"ps":"/wdl_infer/model/ps.json"}}
I0602 02:59:55.921188 549 hugectr.cc:344] *****Parsing Parameter Server Configuration from /wdl_infer/model/ps.json
I0602 02:59:55.921241 549 hugectr.cc:365] Support 64-bit keys = 1
I0602 02:59:55.921276 549 hugectr.cc:583] Model name = wdl
I0602 02:59:55.921282 549 hugectr.cc:592] Model 'wdl' -> network file = /wdl_infer/model/wdl/1/wdl.json
I0602 02:59:55.921290 549 hugectr.cc:599] Model 'wdl' -> max. batch size = 64
I0602 02:59:55.921295 549 hugectr.cc:605] Model 'wdl' -> dense model file = /wdl_infer/model/wdl/1/wdl_dense_20000.model
I0602 02:59:55.921302 549 hugectr.cc:611] Model 'wdl' -> sparse model files = [/wdl_infer/model/wdl/1/wdl0_sparse_20000.model, /wdl_infer/model/wdl/1/wdl1_sparse_20000.model]
I0602 02:59:55.921309 549 hugectr.cc:622] Model 'wdl' -> use GPU embedding cache = 1
I0602 02:59:55.921330 549 hugectr.cc:631] Model 'wdl' -> hit rate threshold = 0.9
I0602 02:59:55.921336 549 hugectr.cc:639] Model 'wdl' -> per model GPU cache = 0.5
[HCTR][02:59:55][WARNING][RK0][main]: default_value_for_each_table.size() is not equal to the number of embedding tables
I0602 02:59:55.921374 549 hugectr.cc:655] Model 'wdl' -> num. pool worker buffers = 4
I0602 02:59:55.921380 549 hugectr.cc:662] Model 'wdl' -> num. pool refresh buffers = 1
I0602 02:59:55.921386 549 hugectr.cc:669] Model 'wdl' -> cache refresh rate per iteration = 0.2
I0602 02:59:55.921393 549 hugectr.cc:678] Model 'wdl' -> deployed device list = [0]
I0602 02:59:55.921400 549 hugectr.cc:686] Model 'wdl' -> default value for each table = [0, 0]
I0602 02:59:55.921408 549 hugectr.cc:706] *****The HugeCTR Backend Parameter Server is creating... *****
[HCTR][02:59:55][INFO][RK0][main]: default_emb_vec_value is not specified using default: 0
[HCTR][02:59:55][INFO][RK0][main]: default_emb_vec_value is not specified using default: 0
I0602 02:59:55.921561 549 hugectr.cc:714] ***** Parameter Server(Int64) is creating... *****
[HCTR][02:59:55][INFO][RK0][main]: Creating ParallelHashMap CPU database backend...
[HCTR][02:59:55][INFO][RK0][main]: Created parallel (16 partitions) blank database backend in local memory!
[HCTR][02:59:55][INFO][RK0][main]: Volatile DB: initial cache rate = 1
[HCTR][02:59:55][INFO][RK0][main]: Volatile DB: cache missed embeddings = 0
[HCTR][02:59:55][DEBUG][RK0][main]: ParallelHashMap backend. Table: hps_et.wdl.sparse_embedding2. Inserted 268619 / 268619 pairs.
[HCTR][02:59:55][INFO][RK0][main]: Table: hps_et.wdl.sparse_embedding2; cached 268619 / 268619 embeddings in volatile database (ParallelHashMap); load: 268619 / 18446744073709551615 (0.00%).
[HCTR][02:59:56][DEBUG][RK0][main]: ParallelHashMap backend. Table: hps_et.wdl.sparse_embedding1. Inserted 1869965 / 1869965 pairs.
[HCTR][02:59:56][INFO][RK0][main]: Table: hps_et.wdl.sparse_embedding1; cached 1869965 / 1869965 embeddings in volatile database (ParallelHashMap); load: 1869965 / 18446744073709551615 (0.00%).
[HCTR][02:59:56][DEBUG][RK0][main]: Real-time subscribers created!
[HCTR][02:59:56][INFO][RK0][main]: Create embedding cache in device 0.
[HCTR][02:59:56][INFO][RK0][main]: Use GPU embedding cache: True, cache size percentage: 0.500000
[HCTR][02:59:56][INFO][RK0][main]: Configured cache hit rate threshold: 0.900000
I0602 02:59:56.205984 549 hugectr.cc:725] *****The HugeCTR Backend Backend created the Parameter Server successfully! *****
I0602 02:59:56.206050 549 hugectr.cc:1703] TRITONBACKEND_ModelInitialize: wdl (version 1)
I0602 02:59:56.206056 549 hugectr.cc:1716] Repository location: /wdl_infer/model/wdl
I0602 02:59:56.206062 549 hugectr.cc:1731] backend configuration in mode: {"cmdline":{"ps":"/wdl_infer/model/ps.json"}}
I0602 02:59:56.207056 549 hugectr.cc:974] Verifying model configuration: {
    "name": "wdl",
    "platform": "",
    "backend": "hugectr",
    "version_policy": {
        "latest": {
            "num_versions": 1
        }
    },
    "max_batch_size": 64,
    "input": [
        {
            "name": "DES",
            "data_type": "TYPE_FP32",
            "format": "FORMAT_NONE",
            "dims": [
                -1
            ],
            "is_shape_tensor": false,
            "allow_ragged_batch": false,
            "optional": false
        },
        {
            "name": "CATCOLUMN",
            "data_type": "TYPE_INT64",
            "format": "FORMAT_NONE",
            "dims": [
                -1
            ],
            "is_shape_tensor": false,
            "allow_ragged_batch": false,
            "optional": false
        },
        {
            "name": "ROWINDEX",
            "data_type": "TYPE_INT32",
            "format": "FORMAT_NONE",
            "dims": [
                -1
            ],
            "is_shape_tensor": false,
            "allow_ragged_batch": false,
            "optional": false
        }
    ],
    "output": [
        {
            "name": "OUTPUT0",
            "data_type": "TYPE_FP32",
            "dims": [
                -1
            ],
            "label_filename": "",
            "is_shape_tensor": false
        }
    ],
    "batch_input": [],
    "batch_output": [],
    "optimization": {
        "priority": "PRIORITY_DEFAULT",
        "input_pinned_memory": {
            "enable": true
        },
        "output_pinned_memory": {
            "enable": true
        },
        "gather_kernel_buffer_threshold": 0,
        "eager_batching": false
    },
    "instance_group": [
        {
            "name": "wdl_0",
            "kind": "KIND_GPU",
            "count": 1,
            "gpus": [
                0
            ],
            "secondary_devices": [],
            "profile": [],
            "passive": false,
            "host_policy": ""
        }
    ],
    "default_model_filename": "",
    "cc_model_filenames": {},
    "metric_tags": {},
    "parameters": {
        "max_nnz": {
            "string_value": "2"
        },
        "embedding_vector_size": {
            "string_value": "128"
        },
        "gpucacheper": {
            "string_value": "0.5"
        },
        "des_feature_num": {
            "string_value": "13"
        },
        "hit_rate_threshold": {
            "string_value": "0.8"
        },
        "gpucache": {
            "string_value": "true"
        },
        "embeddingkey_long_type": {
            "string_value": "true"
        },
        "slots": {
            "string_value": "28"
        },
        "config": {
            "string_value": "/wdl_infer/model/wdl/1/wdl.json"
        },
        "cat_feature_num": {
            "string_value": "28"
        },
        "label_dim": {
            "string_value": "1"
        }
    },
    "model_warmup": []
}
I0602 02:59:56.207229 549 hugectr.cc:1060] The model configuration: {
    "name": "wdl",
    "platform": "",
    "backend": "hugectr",
    "version_policy": {
        "latest": {
            "num_versions": 1
        }
    },
    "max_batch_size": 64,
    "input": [
        {
            "name": "DES",
            "data_type": "TYPE_FP32",
            "format": "FORMAT_NONE",
            "dims": [
                -1
            ],
            "is_shape_tensor": false,
            "allow_ragged_batch": false,
            "optional": false
        },
        {
            "name": "CATCOLUMN",
            "data_type": "TYPE_INT64",
            "format": "FORMAT_NONE",
            "dims": [
                -1
            ],
            "is_shape_tensor": false,
            "allow_ragged_batch": false,
            "optional": false
        },
        {
            "name": "ROWINDEX",
            "data_type": "TYPE_INT32",
            "format": "FORMAT_NONE",
            "dims": [
                -1
            ],
            "is_shape_tensor": false,
            "allow_ragged_batch": false,
            "optional": false
        }
    ],
    "output": [
        {
            "name": "OUTPUT0",
            "data_type": "TYPE_FP32",
            "dims": [
                -1
            ],
            "label_filename": "",
            "is_shape_tensor": false
        }
    ],
    "batch_input": [],
    "batch_output": [],
    "optimization": {
        "priority": "PRIORITY_DEFAULT",
        "input_pinned_memory": {
            "enable": true
        },
        "output_pinned_memory": {
            "enable": true
        },
        "gather_kernel_buffer_threshold": 0,
        "eager_batching": false
    },
    "instance_group": [
        {
            "name": "wdl_0",
            "kind": "KIND_GPU",
            "count": 1,
            "gpus": [
                0
            ],
            "secondary_devices": [],
            "profile": [],
            "passive": false,
            "host_policy": ""
        }
    ],
    "default_model_filename": "",
    "cc_model_filenames": {},
    "metric_tags": {},
    "parameters": {
        "max_nnz": {
            "string_value": "2"
        },
        "embedding_vector_size": {
            "string_value": "128"
        },
        "gpucacheper": {
            "string_value": "0.5"
        },
        "des_feature_num": {
            "string_value": "13"
        },
        "hit_rate_threshold": {
            "string_value": "0.8"
        },
        "gpucache": {
            "string_value": "true"
        },
        "embeddingkey_long_type": {
            "string_value": "true"
        },
        "slots": {
            "string_value": "28"
        },
        "config": {
            "string_value": "/wdl_infer/model/wdl/1/wdl.json"
        },
        "cat_feature_num": {
            "string_value": "28"
        },
        "label_dim": {
            "string_value": "1"
        }
    },
    "model_warmup": []
}
I0602 02:59:56.207347 549 hugectr.cc:1105] slots set = 28
I0602 02:59:56.207354 549 hugectr.cc:1111] desene number = 13
I0602 02:59:56.207360 549 hugectr.cc:1117] cat_feature number = 28
I0602 02:59:56.207366 549 hugectr.cc:1129] embedding size = 128
I0602 02:59:56.207372 549 hugectr.cc:1135] maxnnz = 2
I0602 02:59:56.207378 549 hugectr.cc:1153] HugeCTR model config path = /wdl_infer/model/wdl/1/wdl.json
I0602 02:59:56.207386 549 hugectr.cc:1176] support gpu cache = 1
I0602 02:59:56.207403 549 hugectr.cc:1199] gpu cache per = 0.5
I0602 02:59:56.207409 549 hugectr.cc:1216] hit-rate threshold = 0.8
I0602 02:59:56.207415 549 hugectr.cc:1232] Label dim = 1
I0602 02:59:56.207421 549 hugectr.cc:1238] support 64-bit embedding key = 1
I0602 02:59:56.207426 549 hugectr.cc:1252] Model_Inference_Para.max_batchsize: 64
I0602 02:59:56.207432 549 hugectr.cc:1256] max_batch_size in model config.pbtxt is 64
I0602 02:59:56.207439 549 hugectr.cc:1326] ******Creating Embedding Cache for model wdl in device 0
I0602 02:59:56.207445 549 hugectr.cc:1353] ******Creating Embedding Cache for model wdl successfully
I0602 02:59:56.215901 549 hugectr.cc:1851] TRITONBACKEND_ModelInstanceInitialize: wdl_0 (device 0)
I0602 02:59:56.215912 549 hugectr.cc:1495] Triton Model Instance Initialization on device 0
I0602 02:59:56.256847 549 hugectr.cc:1505] Dense Feature buffer allocation: 
I0602 02:59:56.266345 549 hugectr.cc:1512] Categorical Feature buffer allocation: 
I0602 02:59:56.266456 549 hugectr.cc:1530] Categorical Row Index buffer allocation: 
I0602 02:59:56.266491 549 hugectr.cc:1540] Predict result buffer allocation: 
I0602 02:59:56.266524 549 hugectr.cc:1864] ******Loading HugeCTR Model******
I0602 02:59:56.266531 549 hugectr.cc:1558] The model origin json configuration file path is: /wdl_infer/model/wdl/1/wdl.json
[HCTR][02:59:56][INFO][RK0][main]: Global seed is 3783377676
[HCTR][02:59:57][WARNING][RK0][main]: Peer-to-peer access cannot be fully enabled.
[HCTR][02:59:57][INFO][RK0][main]: Start all2all warmup
[HCTR][02:59:57][INFO][RK0][main]: End all2all warmup
[HCTR][02:59:57][INFO][RK0][main]: Create inference session on device: 0
[HCTR][02:59:57][INFO][RK0][main]: Model name: wdl
[HCTR][02:59:57][INFO][RK0][main]: Use mixed precision: False
[HCTR][02:59:57][INFO][RK0][main]: Use cuda graph: True
[HCTR][02:59:57][INFO][RK0][main]: Max batchsize: 64
[HCTR][02:59:57][INFO][RK0][main]: Use I64 input key: True
[HCTR][02:59:57][INFO][RK0][main]: start create embedding for inference
[HCTR][02:59:57][INFO][RK0][main]: sparse_input name wide_data
[HCTR][02:59:57][INFO][RK0][main]: sparse_input name deep_data
[HCTR][02:59:57][INFO][RK0][main]: create embedding for inference success
[HCTR][02:59:57][INFO][RK0][main]: Inference stage skip BinaryCrossEntropyLoss layer, replaced by Sigmoid layer
I0602 02:59:58.134157 549 hugectr.cc:1565] ******Loading HugeCTR model successfully
I0602 02:59:58.134444 549 model_repository_manager.cc:1152] successfully loaded 'wdl' version 1
I0602 02:59:58.134605 549 server.cc:524] 
+------------------+------+
| Repository Agent | Path |
+------------------+------+
+------------------+------+

I0602 02:59:58.136089 549 server.cc:551] 
+---------+----------------------------------------------------------+-----------------------------------------------+
| Backend | Path                                                     | Config                                        |
+---------+----------------------------------------------------------+-----------------------------------------------+
| hugectr | /usr/local/hugectr/backends/hugectr/libtriton_hugectr.so | {"cmdline":{"ps":"/wdl_infer/model/ps.json"}} |
+---------+----------------------------------------------------------+-----------------------------------------------+

I0602 02:59:58.136182 549 server.cc:594] 
+-------+---------+--------+
| Model | Version | Status |
+-------+---------+--------+
| wdl   | 1       | READY  |
+-------+---------+--------+

I0602 02:59:58.310170 549 metrics.cc:651] Collecting metrics for GPU 0: NVIDIA A100-SXM4-80GB
I0602 02:59:58.310214 549 metrics.cc:651] Collecting metrics for GPU 1: NVIDIA A100-SXM4-80GB
I0602 02:59:58.310224 549 metrics.cc:651] Collecting metrics for GPU 2: NVIDIA A100-SXM4-80GB
I0602 02:59:58.310234 549 metrics.cc:651] Collecting metrics for GPU 3: NVIDIA A100-SXM4-80GB
I0602 02:59:58.310243 549 metrics.cc:651] Collecting metrics for GPU 4: NVIDIA A100-SXM4-80GB
I0602 02:59:58.310253 549 metrics.cc:651] Collecting metrics for GPU 5: NVIDIA A100-SXM4-80GB
I0602 02:59:58.310262 549 metrics.cc:651] Collecting metrics for GPU 6: NVIDIA A100-SXM4-80GB
I0602 02:59:58.310271 549 metrics.cc:651] Collecting metrics for GPU 7: NVIDIA A100-SXM4-80GB
I0602 02:59:58.315118 549 tritonserver.cc:1962] 
+----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Option                           | Value                                                                                                                                                                                  |
+----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| server_id                        | triton                                                                                                                                                                                 |
| server_version                   | 2.20.0                                                                                                                                                                                 |
| server_extensions                | classification sequence model_repository model_repository(unload_dependents) schedule_policy model_configuration system_shared_memory cuda_shared_memory binary_tensor_data statistics |
|                                  |  trace                                                                                                                                                                                 |
| model_repository_path[0]         | /wdl_infer/model/                                                                                                                                                                      |
| model_control_mode               | MODE_EXPLICIT                                                                                                                                                                          |
| startup_models_0                 | wdl                                                                                                                                                                                    |
| strict_model_config              | 1                                                                                                                                                                                      |
| rate_limit                       | OFF                                                                                                                                                                                    |
| pinned_memory_pool_byte_size     | 268435456                                                                                                                                                                              |
| cuda_memory_pool_byte_size{0}    | 67108864                                                                                                                                                                               |
| cuda_memory_pool_byte_size{1}    | 67108864                                                                                                                                                                               |
| cuda_memory_pool_byte_size{2}    | 67108864                                                                                                                                                                               |
| cuda_memory_pool_byte_size{3}    | 67108864                                                                                                                                                                               |
| cuda_memory_pool_byte_size{4}    | 67108864                                                                                                                                                                               |
| cuda_memory_pool_byte_size{5}    | 67108864                                                                                                                                                                               |
| cuda_memory_pool_byte_size{6}    | 67108864                                                                                                                                                                               |
| cuda_memory_pool_byte_size{7}    | 67108864                                                                                                                                                                               |
| response_cache_byte_size         | 0                                                                                                                                                                                      |
| min_supported_compute_capability | 6.0                                                                                                                                                                                    |
| strict_readiness                 | 1                                                                                                                                                                                      |
| exit_timeout                     | 30                                                                                                                                                                                     |
+----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

I0602 02:59:58.317667 549 grpc_server.cc:4421] Started GRPCInferenceService at 0.0.0.0:8001
I0602 02:59:58.317944 549 http_server.cc:3113] Started HTTPService at 0.0.0.0:8000
I0602 02:59:58.359119 549 http_server.cc:178] Started Metrics Service at 0.0.0.0:8002

In [22]:
!curl -v localhost:8000/v2/health/ready

*   Trying 127.0.0.1:8003...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 8003 (#0)
> GET /v2/health/ready HTTP/1.1
> Host: localhost:8003
> User-Agent: curl/7.68.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 404 Not Found
< Server: TornadoServer/6.1
< Content-Type: text/html
< Date: Thu, 02 Jun 2022 03:03:01 GMT
< X-Content-Type-Options: nosniff
< Content-Security-Policy: frame-ancestors 'self'; report-uri /api/security/csp-report
< Content-Length: 2957
< Set-Cookie: _xsrf=2|044a8628|2228373cfd3765ef4f764a5af0ce3e0e|1654138981; expires=Sat, 02 Jul 2022 03:03:01 GMT; Path=/
< 
<!DOCTYPE HTML>
<html>

<head>

    <meta charset="utf-8">

    <title>Jupyter Server</title>
    <link id="favicon" rel="shortcut icon" type="image/x-icon" href="/static/favicon.ico?v=50afa725b5de8b00030139d09b38620224d4e7dba47c07ef0e86d4643f30c9bfe6bb7e1a4a1c561aa32834480909a4b6fe7cd1e17f7159330b6b5914bf45a880">
    
    <link rel="stylesheet" href="/static/style/bootstra